# Transfer Learning MNIST

* Train a simple convnet on the MNIST dataset the first 5 digits [0..4].
* Freeze convolutional layers and fine-tune dense layers for the classification of digits [5..9].

### Import MNIST data and create 2 datasets with one dataset having digits from 0 to 4 and other from 5 to 9 

#### Import the mnist dataset from keras datasets

In [1]:
from keras.datasets import mnist

Using TensorFlow backend.


In [0]:
(trainX, trainY),(testX, testY) = mnist.load_data()

#### Creating two datasets one with digits below 5 and one with 5 and above

In [0]:
x_train_lt5 = trainX[trainY < 5]
y_train_lt5 = trainY[trainY < 5]
x_test_lt5 = testX[testY < 5]
y_test_lt5 = testY[testY < 5]

x_train_gt5 = trainX[trainY >= 5]
y_train_gt5 = trainY[trainY >= 5] - 5  # make classes start at 0 for
x_test_gt5 = testX[testY >= 5]         # np_utils.to_categorical
y_test_gt5 = testY[testY >= 5] - 5

### Check 

Verify shapes of x_train, y_train, x_test and y_test for both the datasets with the below given shapes.

In [61]:
print(x_train_lt5.shape)
print(y_train_lt5.shape)
print(x_test_lt5.shape)
print(y_test_lt5.shape)

(30596, 28, 28)
(30596,)
(5139, 28, 28)
(5139,)


In [62]:
print(x_train_gt5.shape)
print(y_train_gt5.shape)
print(x_test_gt5.shape)
print(y_test_gt5.shape)

(29404, 28, 28)
(29404,)
(4861, 28, 28)
(4861,)


### Let us take only the dataset (x_train, y_train, x_test, y_test) for Integers 0 to 4 in MNIST
### Reshape x_train and x_test to a 4 Dimensional array (channel = 1) to pass it into a Conv2D layer

In [0]:
# input image dimensions
img_rows, img_cols = 28, 28


#Keras expects data to be in the format (N_E.N_H,N_W,N_C)  N_E = Number of Examples, N_H = height, N_W = Width, N_C = Number of Channels.
x_train = x_train_lt5.reshape(x_train_lt5.shape[0], img_rows, img_cols, 1)
x_test = x_test_lt5.reshape(x_test_lt5.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

### Change into float32 datatype and Normalize x_train and x_test by dividing it by 255.0

In [64]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#Normalizing the input
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples


### Check

Verify the shapes of the X_train and X_test with the shapes given below.

In [65]:
print('X_train shape:', x_train.shape)
print('X_test shape:', x_test.shape)

X_train shape: (30596, 28, 28, 1)
X_test shape: (5139, 28, 28, 1)


### Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
import keras
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train_lt5, 5)
y_test = keras.utils.to_categorical(y_test_lt5, 5)

### Build a sequential model with 2 Convolutional layers with 32 kernels of size (3,3) followed by a Max pooling layer of size (2,2) followed by a drop out layer to be trained for classification of digits 0-4  

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [0]:

#Initialize the model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape,name='conv_1'))
model.add(Conv2D(32, (3, 3), activation='relu',name='conv_2'))

#Add a MaxPooling Layer of size 2X2 
model.add(MaxPooling2D(pool_size=(2, 2),name='max_1'))

#Apply Dropout with 0.25 probability 
model.add(Dropout(0.25,name='drop_1'))

### Post that flatten the data and add 2 Dense layers with 128 neurons and neurons = output classes with activation = 'relu' and 'softmax' respectively. Add dropout layer inbetween if necessary  

In [0]:

#Flatten the layer
model.add(Flatten())

#Add Fully Connected Layer with 128 units and activation function as 'ReLU'
model.add(Dense(128, activation='relu',name='dense_1'))
model.add(Dropout(0.5,name='drop_2'))
#Add Fully Connected Layer with 10 units and activation function as 'softmax'
model.add(Dense(5, activation='softmax',name='dense_2'))

In [70]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv_2 (Conv2D)              (None, 24, 24, 32)        9248      
_________________________________________________________________
max_1 (MaxPooling2D)         (None, 12, 12, 32)        0         
_________________________________________________________________
drop_1 (Dropout)             (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               589952    
_________________________________________________________________
drop_2 (Dropout)             (None, 128)              

### Print the training and test accuracy for 5 epochs

In [0]:
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy

#To use adam optimizer for learning weights with learning rate = 0.001
optimizer = Adam(lr=0.001)
#Set the loss function and optimizer for the model training
model.compile(loss=categorical_crossentropy,
              optimizer=optimizer,
              metrics=['accuracy'])

In [72]:
#Training on the dataset
model.fit(x_train, y_train,
          batch_size=128,
          epochs=5,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 2s 64us/step - loss: 0.1421 - acc: 0.9576 - val_loss: 0.0256 - val_acc: 0.9926
Epoch 2/5
30596/30596 [==============================] - 1s 49us/step - loss: 0.0428 - acc: 0.9871 - val_loss: 0.0111 - val_acc: 0.9965
Epoch 3/5
30596/30596 [==============================] - 2s 49us/step - loss: 0.0289 - acc: 0.9918 - val_loss: 0.0084 - val_acc: 0.9973
Epoch 4/5
30596/30596 [==============================] - 1s 49us/step - loss: 0.0219 - acc: 0.9933 - val_loss: 0.0087 - val_acc: 0.9969
Epoch 5/5
30596/30596 [==============================] - 1s 49us/step - loss: 0.0232 - acc: 0.9924 - val_loss: 0.0060 - val_acc: 0.9977


### Use the model trained on 0 to 4 digit classification and train it on the dataset which has digits 5 to 9  (Using Transfer learning keeping only the dense layers to be trainable)

### Make only the dense layers to be trainable and convolutional layers to be non-trainable

#### Check model summary to see model layer names

In [73]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv_2 (Conv2D)              (None, 24, 24, 32)        9248      
_________________________________________________________________
max_1 (MaxPooling2D)         (None, 12, 12, 32)        0         
_________________________________________________________________
drop_1 (Dropout)             (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               589952    
_________________________________________________________________
drop_2 (Dropout)             (None, 128)              

### Do the required preprocessing for `x_train_gt5` also same as `x_train_lt5` and for `y_train_gt5` same as `y_train_lt5`

1. Reshape
2. Change to float32 datatype
3. Normalize (dividing with 255)
4. y_train and y_test Convert into one-hot vectors

In [79]:
img_rows, img_cols = 28, 28


#Keras expects data to be in the format (N_E.N_H,N_W,N_C)  N_E = Number of Examples, N_H = height, N_W = Width, N_C = Number of Channels.
x_train_gt5 = x_train_gt5.reshape(x_train_gt5.shape[0], img_rows, img_cols, 1)
x_test_gt5 = x_test_gt5.reshape(x_test_gt5.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
x_train_gt5 = x_train_gt5.astype('float32')
x_test_gt5 = x_test_gt5.astype('float32')

#Normalizing the input
x_train_gt5 /= 255
x_test_gt5 /= 255
print('x_train shape:', x_train_gt5.shape)
print(x_train_gt5.shape[0], 'train samples')
print(x_test_gt5.shape[0], 'test samples')
import keras
# convert class vectors to binary class matrices
y_train_gt5 = keras.utils.to_categorical(y_train_gt5, 5)
y_test_gt5 = keras.utils.to_categorical(y_test_gt5, 5)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples


### Check

Verify the shapes with the given below.

In [80]:
print(x_train_gt5.shape)
print(y_train_gt5.shape)
print(x_test_gt5.shape)
print(y_test_gt5.shape)

(29404, 28, 28, 1)
(29404, 5)
(4861, 28, 28, 1)
(4861, 5)


## Print the accuracy for classification of digits 5 to 9

In [0]:
model2 = Sequential()

model2.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape,name='conv_1'))
model2.add(Conv2D(32, (3, 3), activation='relu',name='conv_2'))

#Add a MaxPooling Layer of size 2X2 
model2.add(MaxPooling2D(pool_size=(2, 2),name='max_1'))

#Apply Dropout with 0.25 probability 
model2.add(Dropout(0.25,name='drop_1'))
model2.add(Flatten())

#Add Fully Connected Layer with 128 units and activation function as 'ReLU'
model2.add(Dense(128, activation='relu',name='dense_1'))
model2.add(Dropout(0.5,name='drop_2'))
#Add Fully Connected Layer with 10 units and activation function as 'softmax'
model2.add(Dense(5, activation='softmax',name='dense_2'))


In [0]:
for layer in model2.layers:
  if('dense' not in layer.name): #prefix detection to freeze layers which does not have dense
    #Freezing a layer
    layer.trainable = False


In [83]:
#Module to print colourful statements
from termcolor import colored

#Check which layers have been frozen 
for layer in model2.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

conv_1
False
conv_2
False
max_1
False
drop_1
False
flatten_7
False
dense_1
True
drop_2
False
dense_2
True


In [0]:
#Using the Weights of the model trained on digits less than 5.
model2.set_weights(model.get_weights())

In [0]:
model2.compile(loss=categorical_crossentropy,
              optimizer=optimizer,
              metrics=['accuracy'])

In [86]:
#Training on the dataset
model2.fit(x_train_gt5, y_train_gt5,
          batch_size=128,
          epochs=5,
          verbose=1,
          validation_data=(x_test_gt5, y_test_gt5))

Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 1s 49us/step - loss: 0.1765 - acc: 0.9511 - val_loss: 0.0361 - val_acc: 0.9893
Epoch 2/5
29404/29404 [==============================] - 1s 36us/step - loss: 0.0627 - acc: 0.9806 - val_loss: 0.0257 - val_acc: 0.9930
Epoch 3/5
29404/29404 [==============================] - 1s 35us/step - loss: 0.0480 - acc: 0.9851 - val_loss: 0.0275 - val_acc: 0.9905
Epoch 4/5
29404/29404 [==============================] - 1s 35us/step - loss: 0.0429 - acc: 0.9866 - val_loss: 0.0241 - val_acc: 0.9926
Epoch 5/5
29404/29404 [==============================] - 1s 35us/step - loss: 0.0362 - acc: 0.9890 - val_loss: 0.0252 - val_acc: 0.9914


# Text classification using TF-IDF

###  Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### Training data

In [0]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [91]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [92]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [93]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
def tokenize_test(vect):
    X_train_dtm = vect.fit_transform(twenty_train.data)
    print('Features: ', X_train_dtm.shape[1])
    X_test_dtm = vect.transform(twenty_test.data)
    logreg=LogisticRegression()
    logreg.fit(X_train_dtm, twenty_train.target)
    y_train_pred=logreg.predict(X_train_dtm)
    y_pred_class = logreg.predict(X_test_dtm)
    print('Logistic Train Accuracy: ', metrics.accuracy_score(twenty_train.target, y_train_pred))
    print('Logistic Test Accuracy: ', metrics.accuracy_score(twenty_test.target, y_pred_class))

### Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [0]:
from sklearn.feature_extraction.text  import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [96]:
vect = TfidfVectorizer(ngram_range=(1, 1))
tokenize_test(vect)

Features:  35788


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Logistic Train Accuracy:  0.9827204253433761
Logistic Test Accuracy:  0.8868175765645806
